In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/03/09 09:30:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/09 09:30:07 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet

--2025-03-09 08:43:04--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.155.128.187, 18.155.128.46, 18.155.128.6, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.155.128.187|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64346071 (61M) [binary/octet-stream]
Saving to: ‘yellow_tripdata_2024-10.parquet’

yellow_tripdata_202 100%[===================>]  61.36M   167MB/s    in 0.4s    

2025-03-09 08:43:04 (167 MB/s) - ‘yellow_tripdata_2024-10.parquet’ saved [64346071/64346071]



In [3]:
!ls -la

total 376236
drwxrwxr-x  6 almaz.alpyssov almaz.alpyssov      4096 Mar  9 09:29 .
drwxr-xr-x 19 almaz.alpyssov almaz.alpyssov      4096 Mar  7 10:36 ..
drwxrwxr-x  2 almaz.alpyssov almaz.alpyssov      4096 Mar  9 08:41 .ipynb_checkpoints
-rw-rw-r--  1 almaz.alpyssov almaz.alpyssov     21166 Mar  9 09:29 Untitled.ipynb
-rw-rw-r--  1 almaz.alpyssov almaz.alpyssov  11886281 Jul 18  2022 fhv_tripdata_2021-01.parquet
drwxr-xr-x  3 almaz.alpyssov almaz.alpyssov      4096 Mar  9 07:16 fhvhv
-rw-rw-r--  1 almaz.alpyssov almaz.alpyssov 308924937 Jun 30  2022 fhvhv_tripdata_2021-01.parquet
-rw-rw-r--  1 almaz.alpyssov almaz.alpyssov     35956 Mar  9 08:15 hello_spark.ipynb
drwxr-xr-x  2 almaz.alpyssov almaz.alpyssov      4096 Mar  9 08:49 oct2024
-rw-rw-r--  1 almaz.alpyssov almaz.alpyssov     12331 Feb 22  2024 taxi_zone_lookup.csv
-rw-rw-r--  1 almaz.alpyssov almaz.alpyssov  64346071 Dec 18 21:21 yellow_tripdata_2024-10.parquet
drwxr-xr-x  2 almaz.alpyssov almaz.alpyssov      4096 Mar  7 12:48

In [4]:
parqD = spark.read.parquet("yellow_tripdata_2024-10.parquet")
parqD.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-10-01 00:30:44|  2024-10-01 00:48:26|              1|          3.0|         1|                 N|         162|         246|           1|       18.4|  1.0|    0.5|       1.

In [8]:
parqD.repartition(4).write.parquet('oct2024')

In [5]:
parqD.createOrReplaceTempView("parquetFile")

In [15]:
rides = spark.sql("select count(*) from (select VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee from parquetFile where cast(tpep_pickup_datetime as date) = '2024-10-15' group by VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee) as t1")

In [16]:
rides.show()

+--------+
|count(1)|
+--------+
|  128893|
+--------+



# rides_1 = spark.sql("select EXTRACT(EPOCH FROM (tpep_dropoff_datetime-tpep_pickup_datetime ))/3600 from parquetFile limit 100")
rides_1 = spark.sql("select max(((bigint(to_timestamp(tpep_dropoff_datetime)))-(bigint(to_timestamp(tpep_pickup_datetime))))/(3600)) as diff from parquetFile limit 100")
rides_1.show()

In [32]:
rides = spark.sql("select PULocationID, count(*) from parquetFile group by 1 order by 2")
rides.show()

+------------+--------+
|PULocationID|count(1)|
+------------+--------+
|         105|       1|
|         199|       2|
|           5|       2|
|           2|       3|
|         111|       3|
|          44|       4|
|         187|       4|
|          84|       4|
|         204|       4|
|         245|       4|
|          59|       6|
|         109|       6|
|         156|       7|
|         118|       7|
|         206|       9|
|         176|       9|
|          30|      10|
|         172|      10|
|         251|      12|
|         184|      12|
+------------+--------+
only showing top 20 rows

